In [1]:
from datetime import datetime, timedelta
import psycopg2
import pandas as pd
from datetime import datetime, timedelta
from bson import ObjectId
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from sshtunnel import SSHTunnelForwarder
from dbutils.pooled_db import PooledDB

# 전체 대출 정보

In [2]:
ssh_host = 'api.trustyapp.io'
ssh_username = 'ubuntu'
ssh_pem_file = 'ng-prod.pem'
database_host = '127.0.0.1'  # SSH 터널을 통해 localhost로 접근
database_name = 'trusty'
database_username = 'server'
database_password = 'ygFAhsyTMDzB21VdLTnoq7djxwjDNzuc9yPGukm73otXER8ihD'

server = SSHTunnelForwarder(
    (ssh_host, 22),  # SSH 서버 주소와 포트
    ssh_username=ssh_username,
    ssh_pkey=ssh_pem_file,  # Private key 파일 경로
    remote_bind_address=(database_host, 5432)  # 원격 MySQL 서버 주소와 포트
)

server.start()

conn = psycopg2.connect(
    host='127.0.0.1',  # localhost를 통해 SSH 터널로 접속
    user=database_username,
    password=database_password,
    database=database_name,
    port=server.local_bind_port  # sshtunnel이 자동으로 할당한 포트
)
cursor = conn.cursor()

In [3]:
# 대출 정보
total_loan_qry = """
                SELECT "createdAt", "id", "interest", "amount", "status", "repaymentDueDate", "repaidAt", "userId", "bankAccountId" 
                FROM "loan"
                """

cursor.execute(total_loan_qry)
results = cursor.fetchall()

df_loan_total = pd.DataFrame(results)
df_loan_total.columns = [
    "createdAt", "id","interest", "amount", "status", "repaymentDueDate", "repaidAt", "userId", "bankAccountId"
]

#==================
#유저 정보 추가
### user 테이블에서 필요한 컬럼을 선택하여 데이터프레임 생성
user_query = """
    SELECT "id", "lastName", "firstName", "bvn", "gender", "birthDate", "address", "cbKey", "csChannelId"
    FROM "user"
    WHERE "bvn" IS NOT NULL 
"""
cursor.execute(user_query)
user_results = cursor.fetchall()

In [4]:

### user 데이터프레임 생성
df_user = pd.DataFrame(user_results, columns=['id', 'lastName', 'firstName', 'bvn', 'gender', 'birthDate', 'address', 'cbKey', 'csChannelId'])

# df_loan_total과 df_user를 userId를 기준으로 병합
df_loan_total = df_loan_total.merge(df_user, left_on='userId', right_on='id', how='left')

### cbKey 컬럼을 int로 변환
df_loan_total['cbKey'] = df_loan_total['cbKey'].astype(int, errors='ignore')
df_loan_total.drop(columns=['id_y'], inplace=True)
df_loan_total.drop(columns=['id_x'], inplace=True)


#====================
#은행정보 추가
# bank_account 테이블에서 accountNumber 컬럼을 선택하여 데이터프레임 생성
bank_account_query = """
    SELECT "id", "accountNumber"
    FROM bank_account
"""
cursor.execute(bank_account_query)
bank_account_results = cursor.fetchall()

# bank_account 데이터프레임 생성
df_bank_account = pd.DataFrame(bank_account_results, columns=['id', 'accountNumber'])

# df_loan_total과 df_bank_account를 bankAccountId를 기준으로 병합
df_loan_total = df_loan_total.merge(df_bank_account, left_on='bankAccountId', right_on='id', how='left')
df_loan_total.drop(columns=['id'], inplace=True)


In [5]:
# 쿼리 다 쓰면 ssh 터널 닫기
conn.close()
server.stop()

In [6]:
df_loan_total

,createdAt,interest,amount,status,repaymentDueDate,repaidAt,userId,bankAccountId,lastName,firstName,bvn,gender,birthDate,address,cbKey,csChannelId,accountNumber
0,2024-10-30 08:36:50.120390+00:00,1,10,success,2024-11-13 23:00:00+00:00,2024-10-31 08:10:15.168000+00:00,5,22,Adeyemi,Kayode,22324835070,Male,1992-12-13,4 Imam Dauda Street Isuti Road Egan,3,2349169982478,7061100721
1,2024-11-14 08:01:13.212873+00:00,250,5000,success,2024-11-28 23:00:00+00:00,2024-11-25 07:45:59.639000+00:00,11,25,Akinyele,Mustopha,22402486954,Male,1997-09-27,3 opeloyeru street ikotun egbe,1,2349044965581,9044965581
2,2025-01-06 21:43:27.503147+00:00,250,5000,success,2025-01-20 23:00:00+00:00,2025-01-19 13:25:13.745000+00:00,491,417,Isa,Zakari,22492862247,Male,2000-09-27,BLOCK 15 FLAT 7 OSHODI STREET DOLPHIN ESTATE I...,12,2348065283799,8065283799
3,2024-11-23 21:46:05.628470+00:00,250,5000,success,2024-12-07 23:00:00+00:00,2024-12-05 07:30:18.852000+00:00,134,119,Onoshakpokae,Martins,22215647140,Male,1995-04-11,206 IKOTUN ROAD CALVARY,4,2348142889460,8142889460
4,2024-12-26 07:05:53.636320+00:00,250,5000,success,2025-01-09 23:00:00+00:00,NaT,446,369,Oguntoyinbo,Sulaimon,22204235868,Male,1986-04-13,12 musa oyinbo,10,2348085557219,8085557219
5,2025-01-08 03:45:38.428078+00:00,250,5000,success,2025-01-22 23:00:00+00:00,NaT,506,437,Okorie,Maduabuchi,22297991913,Male,1974-08-22,None,22,2349033705225,9033705225
6,2025-01-13 07:10:39.142965+00:00,250,5000,success,2025-01-27 23:00:00+00:00,2025-01-23 07:29:59.055000+00:00,571,516,Omobolanle,Abdulkareem,22475166744,Female,2000-02-29,3 safurat adejumo street igesu ayobo,35,2349029208813,8108718362
7,2025-01-11 01:13:34.199234+00:00,250,5000,success,2025-01-25 23:00:00+00:00,2025-01-31 05:32:58.045000+00:00,488,415,Njoku,Daniel,22144967478,Male,1992-05-03,9 Bolanle Odekoya Owutu Ikorodu Lagos,11,2348131983957,8131983957
8,2025-01-09 13:33:10.177016+00:00,250,5000,success,2025-01-23 23:00:00+00:00,2025-01-23 06:43:37.436000+00:00,552,498,Jamiu,Kamiludeeen,22492778870,Male,1999-09-21,None,23,2348124039599,8124039599
9,2025-01-14 06:22:52.473921+00:00,250,5000,success,2025-01-28 23:00:00+00:00,2025-01-31 11:42:58.917000+00:00,600,540,Dehinbo,Ebenezer,22644848138,Male,2003-11-27,16 OladejoStreet Ikorodu Lagos,38,2347066945524,8107563099


# CRC_1번_시트

In [7]:
# Individual Borrower 데이터프레임 생성
individual_borrower = pd.DataFrame()

# 각 컬럼에 대한 값 설정
individual_borrower['Customer ID'] = df_loan_total['cbKey']
individual_borrower['Branch Code'] = '001'  # 모두 동일
individual_borrower['Surname'] = df_loan_total['lastName']
individual_borrower['First name'] = df_loan_total['firstName']
individual_borrower['Date of Birth'] = df_loan_total['birthDate']
individual_borrower['BVN No'] = df_loan_total['bvn']
individual_borrower['Gender'] = df_loan_total['gender'].apply(lambda x: '001' if x == 'Male' else '002')
individual_borrower['Nationality'] = 'NG'  # 모두 동일
individual_borrower['Primary Address Line 1'] = df_loan_total['address']
individual_borrower['Primary Country'] = 'NG'  # 모두 동일

# BVN No 중복 제거 (첫 번째만 남기고 삭제)
individual_borrower = individual_borrower.drop_duplicates(subset=['BVN No'], keep='first')
individual_borrower['Date of Birth'] = pd.to_datetime(individual_borrower['Date of Birth']).dt.strftime('%d/%m/%Y')

# 결과 확인
individual_borrower.to_csv('crc_individual_borrower.csv')

# CRC_2번_시트

In [8]:
from datetime import datetime

# Credit Information 데이터프레임 생성
credit_information = pd.DataFrame()

# 각 컬럼에 대한 값 설정
credit_information['Customer ID'] = df_loan_total['cbKey']
credit_information['Account Number'] = df_loan_total['accountNumber']
credit_information['Account Status'] = df_loan_total['repaidAt'].isnull().replace({True: 'Open', False: 'Closed'})
credit_information['Loan (Facility) Amount/Availed Limit'] = df_loan_total['amount']
credit_information['Outstanding balance'] = df_loan_total.apply(
    lambda row: 0 if pd.notnull(row['repaidAt']) else row['amount'] + row['interest'], axis=1
)
credit_information['Currency'] = 'NGN'  # 모두 동일

# Days in arrears 계산
credit_information['Days in arrears'] = df_loan_total.apply(
    lambda row: max(0, (datetime.now().date() - row['repaymentDueDate'].date()).days) if pd.isnull(row['repaidAt']) 
    else max(0, (row['repaidAt'].date() - row['repaymentDueDate'].date()).days), axis=1
)

credit_information['Overdue amount'] = df_loan_total.apply(
    lambda row: 0 if pd.notnull(row['repaidAt']) else row['amount'] + row['interest'], axis=1
)
credit_information['Loan (Facility) type'] = '40'  # 모두 동일
credit_information['Maturity date'] = df_loan_total['repaymentDueDate']
credit_information['Loan Classification'] = df_loan_total.apply(
    lambda row: 'Lost' if pd.isnull(row['repaidAt']) and (datetime.now().date() - row['repaymentDueDate'].date()).days > 21 else 'Performing', axis=1
)

# 결과 확인
credit_information.to_csv('crc_credit_information.csv')

# CreditRegistry_1번_시트

In [9]:
# Individual Borrower 데이터프레임 생성
individual_borrower = pd.DataFrame()

# 각 컬럼에 대한 값 설정
individual_borrower['CustomerID'] = df_loan_total['cbKey']
individual_borrower['Surname'] = df_loan_total['lastName']
individual_borrower['First name'] = df_loan_total['firstName']
individual_borrower['Date of Birth'] = df_loan_total['birthDate']
individual_borrower['BVN No'] = df_loan_total['bvn']
individual_borrower['Gender'] = df_loan_total['gender'].apply(lambda x: 'M' if x == 'Male' else 'F')
individual_borrower['Borrower Type'] = 'I'  # 모두 동일

# BVN No 중복 제거 (첫 번째만 남기고 삭제)
individual_borrower = individual_borrower.drop_duplicates(subset=['BVN No'], keep='first')
individual_borrower['Date of Birth'] = pd.to_datetime(individual_borrower['Date of Birth']).dt.strftime('%d/%m/%Y')

individual_borrower.to_csv('CreditRegistry_individual_borrower.csv')

# CreditRegistry_2번_시트

In [10]:
from datetime import datetime

# Credit Information 데이터프레임 생성
credit_information = pd.DataFrame()

# 각 컬럼에 대한 값 설정
credit_information['Customer ID'] = df_loan_total['cbKey']
credit_information['Account Number'] = df_loan_total['accountNumber']
credit_information['Account Status'] = df_loan_total['repaidAt'].isnull().replace({True: 'Open', False: 'Closed'})
credit_information['Account status date'] = df_loan_total['createdAt']
credit_information['Date of loan (facility) disbursement/Loan effective date'] = df_loan_total['createdAt']
credit_information['Credit limit/Facility amount/Global limit'] = ''  # 모두 동일
credit_information['Loan (Facility) Amount/Availed Limit'] = df_loan_total['amount']
credit_information['Outstanding balance'] = df_loan_total.apply(
    lambda row: 0 if pd.notnull(row['repaidAt']) else row['amount'] + row['interest'], axis=1
)
credit_information['Instalment amount'] = ''  # 모두 동일
credit_information['Currency'] = 'NGN'  # 모두 동일
# Days in arrears 계산
credit_information['Days in arrears'] = df_loan_total.apply(
    lambda row: max(0, (datetime.now().date() - row['repaymentDueDate'].date()).days) if pd.isnull(row['repaidAt']) 
    else max(0, (row['repaidAt'].date() - row['repaymentDueDate'].date()).days), axis=1
)
credit_information['Overdue amount'] = df_loan_total.apply(
    lambda row: 0 if pd.notnull(row['repaidAt']) else row['amount'] + row['interest'], axis=1
)
credit_information['Loan (Facility) type'] = 'Personal Loan'  # 모두 동일


#######################
credit_information['Loan (Facility) Tenor'] = 11




credit_information['Repayment frequency'] = '14 days'  # 모두 동일

credit_information['Last payment date'] = df_loan_total.apply(
    lambda row: row['repaidAt'] if pd.notnull(row['repaidAt']) else '', axis=1
)
credit_information['Last payment amount'] =  df_loan_total.apply(
    lambda row: row['amount'] + row['interest'] if pd.notnull(row['repaidAt']) else '', axis=1
)
credit_information['Maturity date'] = df_loan_total['repaymentDueDate']
credit_information['Loan Classification'] = df_loan_total.apply(
    lambda row: 'Lost' if pd.isnull(row['repaidAt']) and (datetime.now().date() - row['repaymentDueDate'].date()).days > 21 else 'Performing', axis=1
)
credit_information['Legal Challenge Status'] = 'No'  # 모두 동일
credit_information['Litigation Date'] = ''  # 모두 동일
credit_information['Consent Status'] = 'Yes'  # 모두 동일
credit_information['Loan Security Status'] = 'No'  # 모두 동일


# 년/월/일 형식 변경
credit_information['Account status date'] = pd.to_datetime(credit_information['Account status date']).dt.strftime('%d/%m/%Y')
credit_information['Date of loan (facility) disbursement/Loan effective date'] = pd.to_datetime(credit_information['Date of loan (facility) disbursement/Loan effective date']).dt.strftime('%d/%m/%Y')
credit_information['Maturity date'] = pd.to_datetime(credit_information['Maturity date']).dt.strftime('%d/%m/%Y')
credit_information['Last payment date'] = pd.to_datetime(credit_information['Last payment date']).dt.strftime('%d/%m/%Y')

# 결과 확인
credit_information.to_csv('CreditRegistry_credit_information.csv')

# FirstCentral_1번_시트

In [11]:
# Individual Borrower 데이터프레임 생성
individual_borrower = pd.DataFrame()

# 각 컬럼에 대한 값 설정
individual_borrower['CustomerID'] = df_loan_total['cbKey']
individual_borrower['Branch Code'] = ''
individual_borrower['Surname'] = df_loan_total['lastName']
individual_borrower['First name'] = df_loan_total['firstName']
individual_borrower['Date of Birth'] = df_loan_total['birthDate']
individual_borrower['BVN No'] = df_loan_total['bvn']
individual_borrower['Gender'] = df_loan_total['gender']
individual_borrower['Nationality'] = 'NG'
individual_borrower['Marital Status'] = ''
individual_borrower['Mobile number'] = df_loan_total['csChannelId']
individual_borrower['Primary Address Line 1'] = df_loan_total['address']
individual_borrower['Primary Address Line 2'] = ''
individual_borrower['Primary city/LGA'] = ''
individual_borrower['Primary State'] = ''
individual_borrower['Primary Country'] = 'NG'
individual_borrower['Employment Status'] = ''
individual_borrower['Occupation'] = ''
individual_borrower['Business Category'] = ''
individual_borrower['Business Sector'] = ''
individual_borrower['Borrower Type'] = ''  # 모두 동일



# BVN No 중복 제거 (첫 번째만 남기고 삭제)
individual_borrower = individual_borrower.drop_duplicates(subset=['BVN No'], keep='first')
individual_borrower['Date of Birth'] = pd.to_datetime(individual_borrower['Date of Birth']).dt.strftime('%d/%m/%Y')

individual_borrower.to_csv('FirstCentral_individual_borrower.csv')

# FirstCentral_2번_시트

In [12]:
from datetime import datetime

# Credit Information 데이터프레임 생성
credit_information = pd.DataFrame()

# 각 컬럼에 대한 값 설정
credit_information['Customer ID'] = df_loan_total['cbKey']
credit_information['Account Number'] = df_loan_total['accountNumber']
credit_information['Account Status'] = df_loan_total['repaidAt'].isnull().replace({True: '001', False: '002'})
credit_information['Account status date'] = df_loan_total['createdAt']
credit_information['Date of loan (facility) disbursement/Loan effective date'] = df_loan_total['createdAt']
credit_information['Credit limit/Facility amount/Global limit'] = ''  # 모두 동일\
credit_information['Loan (Facility) Amount/Availed Limit'] = df_loan_total['amount']
credit_information['Outstanding balance'] = df_loan_total.apply(
    lambda row: 0 if pd.notnull(row['repaidAt']) else row['amount'] + row['interest'], axis=1
)
credit_information['Currency'] = 'NG'  # 모두 동일

# Days in arrears 계산
credit_information['Days in arrears'] = df_loan_total.apply(
    lambda row: max(0, (datetime.now().date() - row['repaymentDueDate'].date()).days) if pd.isnull(row['repaidAt']) 
    else max(0, (row['repaidAt'].date() - row['repaymentDueDate'].date()).days), axis=1
)

credit_information['Overdue amount'] = df_loan_total.apply(
    lambda row: 0 if pd.notnull(row['repaidAt']) else row['amount'] + row['interest'], axis=1
)
credit_information['Loan (Facility) type'] = 40 # 모두 동일
#######################
credit_information['Loan (Facility) Tenor'] = 11
credit_information['Repayment frequency'] = 2  # 모두 동일
credit_information['Last payment date'] = df_loan_total.apply(
    lambda row: row['repaidAt'] if pd.notnull(row['repaidAt']) else '', axis=1
)
credit_information['Last payment amount'] =  df_loan_total.apply(
    lambda row: row['amount'] + row['interest'] if pd.notnull(row['repaidAt']) else '', axis=1
)
credit_information['Maturity date'] = df_loan_total['repaymentDueDate']
credit_information['Loan Classification'] = df_loan_total.apply(
    lambda row: '005' if pd.isnull(row['repaidAt']) and (datetime.now().date() - row['repaymentDueDate'].date()).days > 21 else '001', axis=1
)


credit_information['Marital Status'] = ''  # 모두 동일
credit_information["Spouse's Name"] = ''  # 모두 동일


credit_information['Legal Challenge Status'] = 'No'  # 모두 동일
credit_information['Litigation Date'] = ''  # 모두 동일
credit_information['Consent Status'] = 'Yes'  # 모두 동일
credit_information['Loan Security Status'] = 'No'  # 모두 동일



# 년/월/일 형식 변경
credit_information['Account status date'] = pd.to_datetime(credit_information['Account status date']).dt.strftime('%d/%m/%Y')
credit_information['Date of loan (facility) disbursement/Loan effective date'] = pd.to_datetime(credit_information['Date of loan (facility) disbursement/Loan effective date']).dt.strftime('%d/%m/%Y')
credit_information['Maturity date'] = pd.to_datetime(credit_information['Maturity date']).dt.strftime('%d/%m/%Y')
credit_information['Last payment date'] = pd.to_datetime(credit_information['Last payment date']).dt.strftime('%d/%m/%Y')


# 결과 확인
credit_information.to_csv('FirstCentral_credit_information.csv')